In [11]:
import pandas as pd
import vk_api
import requests
import json
import random
import gspread
from df2gspread import df2gspread as d2g
from oauth2client.service_account import ServiceAccountCredentials

# VK

## отправка обычного сообщения

In [17]:
# ВК токен
token = open("VK_token.txt").read()
app_token = token

# id чата
chat_id = 1

# id кому отправляем
my_id = 4596223

# инициализация сессии
vk_session = vk_api.VkApi(token=app_token)

# использование методов vk как python
vk = vk_session.get_api()



In [18]:
vk.messages.send(
    #chat_id=chat_id, если надо отправить в чат
    user_id=my_id, # если надо отправить лично себе
    random_id=random.randint(1, 2 ** 31),
    message='Это тестовое сообщение через API')

15

## отправка файла

In [12]:
# указание пути к файлу и название файла в чате
path_to_file = 'C:/Users/Администратор/html practice/Artur.jpg'
file_name = 'Artur.jpg'

upload_url = vk.docs.getMessagesUploadServer(peer_id=my_id)["upload_url"]
file = {'file': (file_name, open(path_to_file, 'rb'))}

# запрос на пост файла  
response = requests.post(upload_url, files=file)

json_data = json.loads(response.text)

saved_file = vk.docs.save(file=json_data['file'], title=file_name)
attachment = 'doc{}_{}'.format(saved_file['doc']['owner_id'], saved_file['doc']['id'])

vk.messages.send(
    chat_id=chat_id,  # id of chat where to send
    random_id=random.randint(1, 2 ** 31),  # random number for message identification
    message='Смотри новое фото',  # message text, optional here
    attachment=attachment)  # attachment name

14

# gSheets

## Вытащить данные с gsheets

In [28]:
# путь к кредам
path_to_credential = 'western-dolphin-357814-d9d65046f86c.json' 

# название таблицы на диске
table_name = 'ss_api_test'

scope = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']

credentials = ServiceAccountCredentials.from_json_keyfile_name(path_to_credential, scope)

gs = gspread.authorize(credentials)
work_sheet = gs.open(table_name)

# выбрать первый лист
sheet1 = work_sheet.get_worksheet(0)

# стянуть все данные с листа
data = sheet1.get_all_values()

# заголовки
headers = data.pop(0)

# создать датафрейм
df = pd.DataFrame(data, columns=headers)

In [29]:
df

,num,name
0,1,Anna
1,2,Timur
2,3,Artur
3,4,Alexey


## Редактировать/добавить значения

In [21]:
sheet1.append_row([4, 'Alexey'])

{'spreadsheetId': '1JITA_6CcD4jMqPzdXNkJlEI_Y3SrPU21vtFEZmXlFWo',
 'tableRange': "'Лист1'!A1:B3",
 'updates': {'spreadsheetId': '1JITA_6CcD4jMqPzdXNkJlEI_Y3SrPU21vtFEZmXlFWo',
  'updatedRange': "'Лист1'!A4:B4",
  'updatedRows': 1,
  'updatedColumns': 2,
  'updatedCells': 2}}

## Загрузить данные в sheets

In [23]:
# куда подгрузить данные
spreadsheet_name = '1JITA_6CcD4jMqPzdXNkJlEI_Y3SrPU21vtFEZmXlFWo' # "id table_name"
sheet = 'лист созданный через API'

# загрузка данных
d2g.upload(df, spreadsheet_name, sheet, credentials=credentials, row_names=False)

<Worksheet 'лист созданный через API' id:1058089418>

## Создание новых таблиц

In [30]:
# указание названия файла и путь
sheet = gs.create('create_by_gsAPI')
spreadsheet_url = "https://docs.google.com/spreadsheets/d/%s" % sheet.id

# предоставление доступа пользователям
my_mail = "mrtimmag@gmail.com"
sheet.share(my_mail, perm_type='user', role='writer')

In [31]:
# ссылка на файл
spreadsheet_url

'https://docs.google.com/spreadsheets/d/1DQHx1D4L-bUs3OcAafUR57t5OwUV8nq6olo1Sk6quLg'

# Yandex Metrika

In [35]:
# url к сервису
url = 'https://api-metrika.yandex.net/stat/v1/data?'

# параметры из документации
visits = "metrics=ym:s:visits&dimensions=ym:s:date&filters=ym:s:date=='2023-01-22'&dimensions=ym:s:isRobot&id=44147844"
url = url + visits

# Get запрос и обработка полученного json
query = requests.get(url)
json_data = json.loads(query.text)

# создание df
visits_data = pd.DataFrame([(
                             i['dimensions'][0]['name'],
                             i['dimensions'][1]['name'],
                             i['metrics'][0]) for i in json_data['data']], 
                           columns=['date', 'visits', 'type'])

visits_data

,date,visits,type
0,2023-01-22,People,351.0
1,2023-01-22,Robots,28.0
